In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import time
import pandas as pd

In [3]:
np.__version__

'1.20.1'

In [4]:
pd.__version__

'1.2.4'

1. generazione della popolazione iniziale **P** con un numero ***size*** di cromosomi **C**
2. calcolo **fitness** su P
3. **ordinamento** per valore di fitness
4. se il cromosoma migliore è una soluzione accettabile ESCI, altrimenti:
5. **crossover**
6. **mutazione**
7. ripeti i passi 2 - 6

In [ ]:
class Chromosome:
    def __init__(self, size=1):
        self._size = size
        self._chromosome = np.random.rand(size)
        self._fit_value = None
        
    def __repr__(self):
        return f"Chromosome at {id(self)}"
    
    @property
    def genes(self):
        return self._chromosome
    
    @property
    def fit_value(self):
        return self._fit_value
    
    @fit_value.setter
    def fit_value(self, value):
        self._fit_value = value
    
    @genes.setter
    def genes(self, new_chromosome):
        self._chromosome = new_chromosome
    
    def mutate_gene(self):
        index = np.random.randint(self._size)
        self._chromosome.flat[index] = np.random.rand(1)[0]

In [227]:
class Population:
    def __init__(self, size=100, n_items=8):
        self._size = size
        self._n_items = n_items
        
    def populate(self):
        """
        n_row = self._size
        n_col = self._n_items
        v_func = np.vectorize(lambda x: Chromosome(n_col))
        self._pop = v_func(np.zeros(n_row)) 
        """
        n_row = self._size
        n_col = self._n_items
        #dtype = [('genes', np.ndarray), ('fit_value', float)]
        dtype = [f"g_{x}" for x in range(n_col)] + ['fit_value']
        v_func = np.vectorize(lambda x: Chromosome(n_col))
    @property
    def pop(self):
        if isinstance(self._pop, np.ndarray):
            return self._pop
        else:
            raise Exception('empty poopulation')
        
    @property
    def fitness(self):
        return self._fitness_func
    
    @fitness.setter
    def fitness(self, func):
        self._fitness_func = np.vectorize(func)
    
    def fit(self):
        self._fitness_func(self.pop)   
        
    def sort(self):
        np.sort(self.pop, order=Chromosome.fit_value)

In [24]:
class Population:
    def __init__(self, size=20, n_items=8):
        self._size = size
        self._n_items = n_items
        self._pop = pd.DataFrame(0, index=np.arange(size), columns=[f"g_{x}" for x in range(n_items)])
        self._best_selected = int(size * 0.1)
        self._n_mutations = int(size * 0.1)
        
    @property
    def pop(self):
        return self._pop
    
    def populate(self, func):
        self._gene_func = func
        self._pop = self._pop.applymap(func)
        
    @property
    def fitness(self):
        return self._fitness_func
    
    @fitness.setter
    def fitness(self, func):
        self._fitness_func = func
        
    def fit(self):
        self.pop['fitvalue'] = self.pop.apply(self.fitness, axis=1)
        self.pop.sort_values(by='fitvalue', ascending=False, inplace=True)
        self.max_fitness = self.pop['fitvalue'].max()
        self.min_fitness = self.pop['fitvalue'].min()
    
    # CROSSOVER
    
    def prepare_to_xover(self):
        # delete fintvalue col
        self.pop.drop('fitvalue', inplace=True, axis=1)
        # scramble the pop
        self.pop = self.pop.sample(frac=1)
        sx = self.pop.iloc[::2]
        dx = self.pop.iloc[1::2]
    
    # MUTATION
    
    def apply_mutate(self):
        df_to_mutate = self.pop.sample(self._n_mutations)
        df_to_mutate.apply(mutate, axis=1)
        
        
def mutate(df):
    size = df.values
    index = np.random.randint(size)

In [25]:
# genero un gene

def f(x):
    return np.random.randint(1,101)

In [26]:
# funzione di fitness

def fitfun(chromosome):
    return np.sum(chromosome)

In [ ]:
def get_split(size):
    return np.random.randint(1, size-1)

def xover(c1, c2, size):
    split = get_split(size)
    concat = np.concatenate
    chromo1, chromo2 = c1.genes, c2.genes
    chromo1, chromo2 = concat((chromo1[:split],chromo2[split:])), concat((chromo2[:split],chromo1[split:]))
    c1.genes = chromo1
    c2.genes = chromo2
    
    return 0

In [27]:
a = Population()

In [28]:
a.populate(f)

In [29]:
a.pop

,g_0,g_1,g_2,g_3,g_4,g_5,g_6,g_7
0,95,19,89,21,38,10,42,14
1,44,72,56,88,15,71,82,47
2,87,84,22,97,29,24,2,87
3,90,45,10,11,98,61,16,73
4,69,96,86,32,59,6,82,40
5,97,15,30,28,7,29,60,5
6,93,87,73,94,24,58,81,77
7,79,61,80,19,56,48,44,83
8,49,2,97,70,6,42,70,6
9,77,66,59,50,100,66,70,10


In [30]:
a.fitness = fitfun

In [31]:
a.fit()

In [32]:
a.pop

,g_0,g_1,g_2,g_3,g_4,g_5,g_6,g_7,fitvalue
6,93,87,73,94,24,58,81,77,587
13,53,86,68,67,79,39,82,57,531
15,39,23,58,40,88,87,81,97,513
9,77,66,59,50,100,66,70,10,498
18,45,58,44,10,99,96,45,84,481
1,44,72,56,88,15,71,82,47,475
4,69,96,86,32,59,6,82,40,470
7,79,61,80,19,56,48,44,83,470
12,76,43,30,90,25,73,88,14,439
2,87,84,22,97,29,24,2,87,432


In [193]:
def get_split(size):
    return np.random.randint(1, size-1)

In [ ]:
# gggggggg -> max dim = 6
# if dim == 4 ->
# abbbbaaa -> aabbbbaa -> aaabbbba
# if dim == 6 -> abbbbbba

In [202]:
def get_split2(size):
    segment = np.random.randint(1, size-2)
    splitpoint = np.random.randint(1, size - segment)
    #print('*' * split + '-' * segment + '*' * (size - segment - split), segment, split)
    return splitpoint, segment

In [204]:
def xover(c1, c2, size):
    split = get_split(size)
    concat = np.concatenate
    chromo1, chromo2 = c1.genes, c2.genes
    chromo1, chromo2 = concat((chromo1[:split],chromo2[split:])), concat((chromo2[:split],chromo1[split:]))
    c1.genes = chromo1
    c2.genes = chromo2
    
    return 0

In [205]:
def xover2(c1, c2, size):
    split, segment = get_split2(size)
    concat = np.concatenate
    chromo1, chromo2 = c1.genes, c2.genes
    chromo1 = concat(chromo1[:split],chromo2[split:split+segment],chromo1[split+segment:])
    chromo2 = concat(chromo2[:split],chromo1[split:split+segment],chromo2[split+segment:])
    c1.genes = chromo1
    c2.genes = chromo2
    
    return 0

In [228]:
p = Population(20, 8)

In [229]:
start = time.time()
p.populate()
time.time() - start

0.0002646446228027344

In [237]:
p.pop

array([Chromosome at 140040014736144, Chromosome at 140040014733840,
       Chromosome at 140040014734096, Chromosome at 140040014734608,
       Chromosome at 140040021364304, Chromosome at 140040021364560,
       Chromosome at 140040021364368, Chromosome at 140040021360784,
       Chromosome at 140040021363472, Chromosome at 140040021364432,
       Chromosome at 140040021360848, Chromosome at 140040021361552,
       Chromosome at 140040021363344, Chromosome at 140040021362832,
       Chromosome at 140040021363152, Chromosome at 140040021363920,
       Chromosome at 140040021363024, Chromosome at 140040021360720,
       Chromosome at 140040021361168, Chromosome at 140040021361680],
      dtype=object)

In [231]:
# funzione di fitness
# la variabile Chromosome.genes non dovrebbe essere chiamata esplicitamente, quindi bisogna trovare 
# una funzione di più alto livello che venga chiamata con i metodi corretti

def fit_fun(chromosome):
    chromosome.fit_value = np.sum(chromosome.genes)

In [ ]:
# ad esempio, la funzione di fitness prende un cromosoma (istanza di Chromosome)

In [232]:
p.fitness = fit_fun

In [233]:
p.fit()

In [234]:
p.pop[2].fit_value

3.907775064077095

In [235]:
p.pop[0].genes

array([0.74387209, 0.8571742 , 0.17250363, 0.57398267, 0.83881516,
       0.85418747, 0.45147309, 0.00174877])

In [191]:
p.pop[0].mutate_gene()

In [192]:
p.pop[0].genes

array([0.11591088, 0.84887496, 0.38702362, 0.89799612, 0.60198784,
       0.70597501, 0.49185147, 0.94902503])

In [174]:
type(p.pop[0])

__main__.Chromosome

In [236]:
p.sort()

ValueError: Cannot specify order when the array has no fields.

In [605]:
SIZE = 8

In [606]:
tastoma = pd.DataFrame(0, index=np.arange(100), columns=[f"g_{x}" for x in range(SIZE)] + ['fitvalue'])

In [607]:
tastoma.iloc[:,:SIZE] = tastoma.iloc[:,:SIZE].applymap(lambda x: np.random.randint(1,11))

In [608]:
tastoma['fitvalue'] = tastoma.iloc[:,:SIZE].apply(lambda x: np.sum(x.values), axis=1)

In [609]:
tastoma.sort_values(by='fitvalue', ascending=False, inplace=True)

In [610]:
tastoma.head()

,g_0,g_1,g_2,g_3,g_4,g_5,g_6,g_7,fitvalue
40,10,8,8,10,5,7,7,10,65
2,10,7,7,6,6,10,6,10,62
21,8,2,9,9,8,7,10,8,61
22,7,5,9,9,10,4,7,9,60
87,6,9,10,8,7,4,7,7,58


In [506]:
def xov(dataframe, i1, i2):
    concat = np.concatenate
    split = get_split(SIZE)
    chromo1, chromo2 = dataframe.iloc[i1,:-1].values, dataframe.iloc[i2,:-1].values
    chromo1, chromo2 = concat((chromo1[:split],chromo2[split:])), concat((chromo2[:split],chromo1[split:]))
    dataframe.iloc[i1,:-1] = chromo1
    dataframe.iloc[i2,:-1] = chromo2
    return None

In [520]:
xov(tastoma, 0, 3)

In [532]:
tastoma

,g_0,g_1,g_2,g_3,g_4,g_5,g_6,g_7,fitvalue
45,8,8,6,8,5,10,9,10,64
59,9,10,10,8,7,10,1,4,59
87,7,8,1,9,7,10,8,9,59
0,10,10,4,3,8,8,5,9,57
85,9,3,7,9,1,10,7,10,56
...,...,...,...,...,...,...,...,...,...
16,4,3,3,2,9,5,4,3,33
50,6,6,4,3,6,2,2,2,31
15,1,6,6,8,1,5,1,2,30
54,4,4,2,1,8,1,6,2,28


In [472]:
c1

g1          10
g2           9
g3          10
fitvalue    29
Name: 35, dtype: int64

In [611]:
sample = tastoma.sample(5)

In [613]:
sample = sample.applymap(lambda x: 0)

In [618]:
sample.drop('fitvalue', inplace=True, axis=1)

In [619]:
sample

,g_0,g_1,g_2,g_3,g_4,g_5,g_6,g_7
97,0,0,0,0,0,0,0,0
44,0,0,0,0,0,0,0,0
25,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0
72,0,0,0,0,0,0,0,0


In [616]:
tastoma.drop('fitvalue', inplace=True, axis=1)

In [620]:
tastoma.loc[sample.index] = sample

In [621]:
tastoma.loc[97]

g_0    0
g_1    0
g_2    0
g_3    0
g_4    0
g_5    0
g_6    0
g_7    0
Name: 97, dtype: int64

In [622]:
tastoma

,g_0,g_1,g_2,g_3,g_4,g_5,g_6,g_7
40,10,8,8,10,5,7,7,10
2,0,0,0,0,0,0,0,0
21,8,2,9,9,8,7,10,8
22,7,5,9,9,10,4,7,9
87,6,9,10,8,7,4,7,7
...,...,...,...,...,...,...,...,...
56,3,3,1,8,6,1,6,2
16,1,6,4,2,10,1,1,4
10,1,4,2,8,1,6,1,4
82,2,3,4,6,4,2,1,1


In [623]:
tastoma['fitvalue'] = tastoma.iloc[:,:SIZE].apply(lambda x: np.sum(x.values), axis=1)

In [624]:
tastoma

,g_0,g_1,g_2,g_3,g_4,g_5,g_6,g_7,fitvalue
40,10,8,8,10,5,7,7,10,65
2,0,0,0,0,0,0,0,0,0
21,8,2,9,9,8,7,10,8,61
22,7,5,9,9,10,4,7,9,60
87,6,9,10,8,7,4,7,7,58
...,...,...,...,...,...,...,...,...,...
56,3,3,1,8,6,1,6,2,30
16,1,6,4,2,10,1,1,4,29
10,1,4,2,8,1,6,1,4,27
82,2,3,4,6,4,2,1,1,23


In [625]:
tastoma.sort_values(by='fitvalue', ascending=False, inplace=True)
tastoma

,g_0,g_1,g_2,g_3,g_4,g_5,g_6,g_7,fitvalue
40,10,8,8,10,5,7,7,10,65
21,8,2,9,9,8,7,10,8,61
22,7,5,9,9,10,4,7,9,60
87,6,9,10,8,7,4,7,7,58
32,10,7,7,4,8,10,9,2,57
...,...,...,...,...,...,...,...,...,...
25,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0
44,0,0,0,0,0,0,0,0,0
97,0,0,0,0,0,0,0,0,0


In [61]:
import pandas as pd
import numpy as np
  
# create a DataFrame
ODI_runs = {'name': ['Tendulkar', 'Sangakkara', 'Ponting',
                      'Jayasurya', 'Jayawardene', 'Kohli',
                      'Haq', 'Kallis', 'Ganguly', 'Dravid'],
            'runs': [18426, 14234, 13704, 13430, 12650,
                     11867, 11739, 11579, 11363, 10889]}
df = pd.DataFrame(ODI_runs)
  
# print the original DataFrame
print("Original DataFrame :")
df

Original DataFrame :


,name,runs
0,Tendulkar,18426
1,Sangakkara,14234
2,Ponting,13704
3,Jayasurya,13430
4,Jayawardene,12650
5,Kohli,11867
6,Haq,11739
7,Kallis,11579
8,Ganguly,11363
9,Dravid,10889


In [84]:
sx = df.iloc[::2]

In [85]:
dx = df.iloc[1::2]

In [86]:
sx, dx

(          name   runs
 0    Tendulkar   1000
 2      Ponting  13704
 4  Jayawardene  12650
 6          Haq  11739
 8      Ganguly  11363,
          name   runs
 1  Sangakkara  14234
 3   Jayasurya  13430
 5       Kohli  11867
 7      Kallis  11579
 9      Dravid  10889)

In [99]:
for i,j in zip(sx.items(),dx.items()):
    print(i[1],j[1])

0      Tendulkar
2        Ponting
4    Jayawardene
6            Haq
8        Ganguly
Name: name, dtype: object 1    Sangakkara
3     Jayasurya
5         Kohli
7        Kallis
9        Dravid
Name: name, dtype: object
0     1000
2    13704
4    12650
6    11739
8    11363
Name: runs, dtype: int64 1    14234
3    13430
5    11867
7    11579
9    10889
Name: runs, dtype: int64


In [97]:
type(i[1])

pandas.core.series.Series

In [90]:
sx.apply(lambda x: print(x))

0      Tendulkar
2        Ponting
4    Jayawardene
6            Haq
8        Ganguly
Name: name, dtype: object
0     1000
2    13704
4    12650
6    11739
8    11363
Name: runs, dtype: int64


name    None
runs    None
dtype: object

In [74]:
df.loc[sx.iloc[0].name, 'runs':] = [1000,2000]

ValueError: Must have equal len keys and value when setting with an iterable

In [73]:
df

,name,runs
0,Tendulkar,1000
1,Sangakkara,14234
2,Ponting,13704
3,Jayasurya,13430
4,Jayawardene,12650
5,Kohli,11867
6,Haq,11739
7,Kallis,11579
8,Ganguly,11363
9,Dravid,10889
